# Build and Train a Model
In this notebook we preprocess the data to get it ready for a model, extend a pre-trained model from Keras, train the model on
the x-ray images in our dataset, and evaluate that models performance.

The goal is to be able to accurately classify pneumonia from only looking at the x_ray image with a reasonable level of accuracy.
Finding a baseline accuracy for this task is more complicated than just considering how often the model correctly classifies
pneumonia vs not pneumonia. Pneumonia is rare in our dataset (1.276%) so a naive model could just classify every image as not pneumonia
do very well (98.724% Accurate). Therefore we will use other metrics like Recall, Precision, and F1 Score to evaluate our model.
This [paper](https://arxiv.org/pdf/1711.05225.pdf) goes into detail on finding a good baseline accuracy. According to this paper
the average radiologist F1 score for detecting pneumonia from x_ray images with no other information about the patients
is 0.387 (based on the radiologist they used for this study). I hope to build a model that can come close to this baseline or
even outperform it.

## Import Necessary Packages

In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import chain
from random import sample
from sklearn import model_selection, metrics
import tensorflow as tf
from skimage import io
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential, Model

## Setup the Data
In this section we setup the data to get it ready for our model. I parse the `Finding Label` to make create binary
classification labels for each type of disease for every observation. I also create a `path` column for each `Image Index`
so I can get to the location of the actual images.

In [ ]:
# Read in the Data
all_xray_df = pd.read_csv('/data/Data_Entry_2017.csv')

# Convert `Finding Labels` to binary labels for each disease
all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
for label in all_labels:
    if len(label)>1:
        all_xray_df[label] = all_xray_df['Finding Labels'].map(lambda diseases: 1.0 if label in diseases else 0)

# Create 'path' column for each 'Image Index'
all_image_paths = {os.path.basename(x): x for x in
                   glob(os.path.join('/data','images*', '*', '*.png'))}
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)

# Check the results
print(f'Images Found: {len(all_image_paths)}, Total Observations: {all_xray_df.shape[0]}')
all_xray_df.sample(3)


## Create Training and Validation Sets
In this section I create the training and validation sets for the model. I am using an 90/10 split of the data with
90% of the positive pneumonia cases in the training set and 10% of the positive pneumonia cases in the validation set.
Also for the training set the proportion of positive to negative pneumonia cases will be 50/50.

In [2]:
def create_splits(df, validation_percent):
    # create original train / validation split
    train_data, val_data = model_selection.train_test_split(df, test_size=validation_percent, stratify=df['Pneumonia'])

    # Update training set to be 50/50 split
    pos_indexes = train_data[train_data.Pneumonia == 1].index.tolist()
    neg_indexes = train_data[train_data.Pneumonia == 0].index.tolist()
    neg_sample = sample(neg_indexes, len(pos_indexes))
    train_data = train_data.loc[pos_indexes+neg_sample]

    # Check splits
    print(f'Total Pneumonia Cases: {df[df.Pneumonia==1].shape[0]} \
    {(1-validation_percent)*100}% Pneumonia Cases: {int(df[df.Pneumonia==1].shape[0]*(1-validation_percent))} \
    {validation_percent*100}% Pneumonia Cases: {int(df[df.Pneumonia==1].shape[0]*validation_percent)}')
    print(f'Pneumonia Cases in Training set: {train_data[train_data.Pneumonia==1].shape[0]} \
    Pneumonia Cases in Validation set: {val_data[val_data.Pneumonia==1].shape[0]}')

    print(f'Train Set Size: {train_data.shape[0]}, \
    Pos %: {train_data[train_data.Pneumonia==1].shape[0] / train_data.shape[0] *100}, \
    Neg %: {train_data[train_data.Pneumonia==0].shape[0] / train_data.shape[0] *100}')

    print(f'Original Set Size: {df.shape[0]} \
    Pos %: {df[df.Pneumonia==1].shape[0] / df.shape[0] *100} \
    Neg %: {df[df.Pneumonia==0].shape[0] / df.shape[0] *100}')

    print(f'Validation Set Size: {val_data.shape[0]}, \
    Pos %: {val_data[val_data.Pneumonia == 1].shape[0] / val_data.shape[0] *100}, \
    Neg % {val_data[val_data.Pneumonia == 0].shape[0] / val_data.shape[0] *100}')

    return train_data, val_data

train_df, val_df = create_splits(all_xray_df, 0.1)


NameError: name 'all_xray_df' is not defined

## Implement Image Augmentation and Setup Data Generators
In this section I set up image augmentation to generate a more diverse training set, and create training and validation
set generators to use for model training and testing.

In [ ]:
IMG_SIZE = (224, 224)

# Create Image Data Generator for training set
train_idg = ImageDataGenerator(rescale=1.0/255.0,
                         horizontal_flip = True,
                         vertical_flip = False,
                         rotation_range = 0.5,
                         shear_range = 0.1,
                         zoom_range = 0.15)

train_generator = train_idg.flow_from_dataframe(dataframe=train_df, directory=None,
                                          x_col='path', y_col='Pneumonia',
                                          class_mode = 'raw', target_size=IMG_SIZE, batch_size=64)

# Create a generator for the test set
validation_idg = ImageDataGenerator(rescale=1.0/255.0)
validation_generator = validation_idg.flow_from_dataframe(dataframe=val_df, directory=None,
                                                      x_col='path', y_col='Pneumonia',
                                                      class_mode='raw', target_size=IMG_SIZE, batch_size=1024,
                                                      shuffle = True)

In [ ]:
# Check a random batch of training data to make sure everything looks okay
# Training Examples
t_x, t_y = next(train_generator)
fig, m_axs = plt.subplots(4, 4, figsize = (16, 16))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone')
    if c_y == 1: 
        c_ax.set_title('Pneumonia')
    else:
        c_ax.set_title('No Pneumonia')
    c_ax.axis('off')


## Extending a Pre-trained Model
In this section I use pre-trained models from Keras to attempt to solve this classification problem. I used the
VGG19 architecture.

In [ ]:
# Function to setup our own model that is an extension of already pre trained model
def setup_model(model, input_shape, output_size: int, dropout: bool, num_active_layers: int, lr):
    # Creat new network based on parameters
    new_model = tf.keras.Sequential()
    new_model.add(tf.keras.layers.Input(shape=input_shape, dtype='float32'))

    # Setup Fully Connected Layers (CURRENTLY HARDCODED FOR VGG19)
    if dropout:
        for layer in model.layers[:-3]:
            new_model.add(layer)
        new_model.add(Dense(4096, activation='relu'))
        new_model.add(Dropout(0.5))
        new_model.add(Dense(4096, activation='relu'))
        new_model.add(Dropout(0.5))
    else:
        for layer in model.layers[:-1]:
            new_model.add(layer)

    # Add in our own output layer
    new_model.add(tf.keras.layers.Dense(output_size,activation='sigmoid'))

    # Freeze all but num_active_layers layers
    for layer_num, layer in enumerate(new_model.layers):
        if layer_num < len(new_model.layers) - num_active_layers:
            layer.trainable = False

    # Setup optimizer, loss function, and metrics
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return new_model

In [ ]:
# Setup Different VGG19 Architecture
vgg19 = tf.keras.applications.VGG19(include_top=True, weights='imagenet')

baseline_model = setup_model(vgg19, input_shape=(224,224,3), output_size=1, dropout=False, num_active_layers=8, lr=1e-4)
baseline_model.summary()

dropout_vgg_model = setup_model(vgg19, input_shape=(224,224,3), output_size=1, dropout=True, num_active_layers=8, lr=1e-4)
dropout_vgg_model.summary()

dropout_lowlr_vgg_model = setup_model(vgg19, input_shape=(224,224,3), output_size=1, dropout=True, num_active_layers=8, lr=1e-5)
dropout_lowlr_vgg_model.summary()

## Setup Model Callbacks
Below is some helper code that will allow us to add callbacks to our model. This will save the 'best' version of the model
by comparing it to previous epochs of training. The 'patience' parameter tells the code how long to wait without seeing
any improvements to the chosen metric before quitting.

In [ ]:
weight_path = f"current_best.hdf5"

checkpoint = tf.keras.callbacks.ModelCheckpoint(weight_path,
                                                monitor= 'val_loss',
                                                verbose=1,
                                                save_best_only=True,
                                                mode= 'min',
                                                save_weights_only = True)

early = tf.keras.callbacks.EarlyStopping(monitor= 'val_loss', mode= 'min', patience=25)

callbacks_list = [checkpoint, early]

## Train and Save the Models
Everything is already set up for this, all we have to do is run the model and look at the results below. I set up the ability
to run all the available models. If you do not want to run any just comment them out. Also note that all models are saved
to the models directory so you can just load in the models if you do not want to rerun them.

In [ ]:
# Baseline Model TODO: SETUP Baseline we originally used
vgg_history = baseline_model.fit(train_generator, validation_data = next(validation_generator), epochs = 100, callbacks = callbacks_list)
baseline_model.load_weights(weight_path)
dropout_vgg_model.save("models/baseline_model.h5")

# Dropout with normal learning rate
dropout_vgg_history = dropout_vgg_model.fit(train_generator, validation_data = next(validation_generator), epochs = 100, callbacks = callbacks_list)
dropout_vgg_model.load_weights(weight_path)
dropout_vgg_model.save("models/dropout_model.h5")

# Dropout with lower learning rate
dropout_lowlr_vgg_history = dropout_lowlr_vgg_model.fit(train_generator, validation_data = next(validation_generator), epochs = 100, callbacks = callbacks_list)
dropout_lowlr_vgg_model.load_weights(weight_path)
dropout_lowlr_vgg_model.save("models/dropout_lowlr_model.h5")

## Evaluate Model Performance
Training the model results in a log of information on training loss, validation loss, training accuracy, and
validation accuracy. You can view these results below. While these are helpful for showing us if our model is learning something,
they are not very helpful when it comes to medical classification problems, especially if the disease you are trying to
classify is very rare in the dataset (which is the case with pneumonia in this dataset). This is still helpful information,
but we will look at better metrics below in the section on `Performance Statistics`.


In [ ]:
# Plot Model History
def plot_history(model_history):
    N = len(model_history.history['loss'])
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0,N), model_history.history['loss'], label='train loss')
    plt.plot(np.arange(0,N), model_history.history['val_loss'], label='valuation loss')
    plt.plot(np.arange(0,N), model_history.history['binary_accuracy'], label='train accuracy')
    plt.plot(np.arange(0,N), model_history.history['val_binary_accuracy'], label='valuation accuracy')
    plt.title("Model Training Results")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss \ Accuracy")
    plt.legend(loc="upper left")
    plt.plot()

plot_history(vgg_history)
plot_history(dropout_vgg_history)
plot_history(dropout_lowlr_vgg_history)



## Look at Performance Statistics
Below we look at some performance statistics, more specifically the ROC curve along with AUC, the precision-recall curve,
and the F1 scores for different thresholds. These statistics will help us decide a correct threshold for our
classification problem.

In [1]:
#-----------         Functions for getting model predictions and plotting results       -------------#
def get_model_predictions(model_filepath, validation_data):
    """
    Takes in a filepath to the saved model, loads in the model, and gets predictions for all data in validation_generator
    :param model_filepath: filepath to a saved model, must have .h5 extension
    :param validation_data: Validation data to make predictions on. Can be anything Model.predict from Keras accepts. (array, tensor, generator, ect)
    :return: a list of predictions for each input
    """
    model = tf.keras.models.load_model(model_filepath)
    pred_y = model.predict(validation_data)
    return [data for answer in pred_y for data in answer]

def plot_roc_curve(model_name, true_y, pred_y):
    """
    Plot the ROC curve along with the curves AUC for a given model. Note make sure true_y and pred_y are from the same model as model_name
    :param model_name: Name of model used for saving plot
    :param true_y: true labels for dataset
    :param pred_y: predicted labels for dataset
    """
    fig, ax = plt.subplots(1,1, figsize=(7,7))
    fpr, tpr, thresholds = metrics.roc_curve(true_y, pred_y)
    ax.plot(fpr, tpr, label=f'{model_name} AUC: {metrics.auc(fpr,tpr)}')
    ax.legend()
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    plt.savefig(f'images/{model_name}_roc.png')
    return

def plot_precision_recall_curve(model_name, true_y, pred_y):
    """
    Plot the precision recall curve for a given model. Note make sure true_y and pred_y are from the same model as model_name
    :param model_name: Name of model used for saving plot
    :param true_y: true labels for dataset
    :param pred_y: predicted labels for dataset
    """
    fig, ax = plt.subplots(1,1, figsize=(7,7))
    precision, recall, thresholds = metrics.precision_recall_curve(true_y, pred_y)
    ax.plot(recall, precision, label=f'{model_name} AP Score: {metrics.average_precision_score(true_y,pred_y)}')
    plt.legend()
    ax.set_xlabel("Recall")
    ax.set_ylabel("Precision")
    plt.savefig(f'images/{model_name}_precision_recall.png')
    return

def calculate_f1_scores(precision, recall):
    return [(2*p*r)/(p+r) for p,r in zip(precision,recall)]

def plot_f1_score(model_name, true_y, pred_y):
    """
    Plot F1 Scores for a given model. Note make sure true_y and pred_y are from the same model as model_name
    F1 = 2*(precision*recall) / (precision + recall)
    :param model_name: Name of model used for saving plot
    :param true_y: true labels for dataset
    :param pred_y: predicted labels for the dataset
    """
    fig, ax = plt.subplots(1,1,figsize=(7,7))
    precision, recall, thresholds = metrics.precision_recall_curve(true_y,pred_y)
    ax.plot(thresholds, precision[:-1], label=f'{model_name} Precision')
    ax.plot(thresholds, recall[:-1], label=f'{model_name} Recall')
    f1_scores = calculate_f1_scores(precision, recall)[:-1]
    ax.plot(thresholds, f1_scores, label=f'{model_name} F1 Score')
    plt.legend()
    ax.set_xlabel("Threshold Values")
    plt.savefig(f'images/{model_name}_f1_score.png')
    return max(f1_scores)

NameError: name 'new_vgg_model' is not defined

In [ ]:
# Generate Plots for Models
validation_generator = validation_idg.flow_from_dataframe(dataframe=val_df, directory=None,
                                                      x_col='path', y_col='Pneumonia',
                                                      class_mode='raw', target_size=IMG_SIZE, batch_size=len(val_df),
                                                      shuffle = False)

# Baseline Model Plots
pred_y = get_model_predictions("models/baseline_model.h5", validation_generator)
true_y = val_df['Pneumonia'].values()
plot_roc_curve('VGG19_baseline', true_y, pred_y)
plot_precision_recall_curve('VGG19_baseline', true_y, pred_y)
plot_f1_score('VGG19_baseline', true_y, pred_y)

In [ ]:
# Dropout Model Plots
pred_y = get_model_predictions("models/dropout_model.h5", validation_generator)
plot_roc_curve('VGG19_dropout', true_y, pred_y)
plot_precision_recall_curve('VGG19_dropout', true_y, pred_y)
plot_f1_score('VGG19_dropout', true_y, pred_y)

In [ ]:
# Dropout with Low LR Model Plots
pred_y = get_model_predictions("models/dropout_lowlr_model.h5", validation_generator)
plot_roc_curve('VGG19_dropout_lowlr', true_y, pred_y)
plot_precision_recall_curve('VGG19_dropout_lowlr', true_y, pred_y)
plot_f1_score('VGG19_dropout_lowlr', true_y, pred_y)


## Decide on classification threshold
It is difficult to decide on a classification threshold for this model because we do not entirely know what the use case
is for our model. If this is a preliminary screening we want high recall so that we do not miss anyone that potentially has the disease.
If the cost of the follow up exam is very expensive we want high precision because we do not want to waste patients money.
Then again if missing a positive case leads to death of the patient we definitely want high recall to err on the side of caution.
We could also just maximize the F1 score if we want to balance precision and recall.

Based on the results from the graphs above I choose a threshold of ... to ...

Look at Example Cases based on Threshold

In [ ]:
#TODO
## Find the threshold that optimize your model's performance,
## and use that threshold to make binary classification. Make sure you take all your metrics into consideration.

In [ ]:
## Let's look at some examples of predicted v. true with our best model: 

# Todo

# fig, m_axs = plt.subplots(10, 10, figsize = (16, 16))
# i = 0
# for (c_x, c_y, c_ax) in zip(valX[0:100], testY[0:100], m_axs.flatten()):
#     c_ax.imshow(c_x[:,:,0], cmap = 'bone')
#     if c_y == 1: 
#         if pred_Y[i] > YOUR_THRESHOLD:
#             c_ax.set_title('1, 1')
#         else:
#             c_ax.set_title('1, 0')
#     else:
#         if pred_Y[i] > YOUR_THRESHOLD: 
#             c_ax.set_title('0, 1')
#         else:
#             c_ax.set_title('0, 0')
#     c_ax.axis('off')
#     i=i+1